In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load pre-trained MobileNetV2 model + higher level layers
model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Function to extract features from a single frame
def extract_features(frame):
    img = image.img_to_array(frame)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = model.predict(img)
    return features

# Function to extract features from a video
def extract_video_features(video_path):
    cap = cv2.VideoCapture(video_path)
    features_list = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Ensure the frame size matches the model's input size
        features = extract_features(frame)
        features_list.append(features)
    cap.release()
    return np.mean(features_list, axis=0).squeeze()  # Average pooling over all frames and squeeze to remove extra dimensions

# Directory containing preprocessed videos
preprocessed_video_dir = '/content/drive/MyDrive/processed_sign_language_dataset'

# List to store extracted features and labels
video_features_list = []
labels = []

# Extract features for each video in each class
for class_folder in os.listdir(preprocessed_video_dir):
    class_folder_path = os.path.join(preprocessed_video_dir, class_folder)
    if os.path.isdir(class_folder_path):
        for filename in os.listdir(class_folder_path):
            if filename.endswith(".mp4"):
                video_path = os.path.join(class_folder_path, filename)
                features = extract_video_features(video_path)
                video_features_list.append(features)
                labels.append(class_folder)

# Convert lists to numpy arrays
X = np.array(video_features_list)
y = np.array(labels)

print("Feature extraction complete.")
print("Shape of extracted features:", X.shape)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 32ms/step
Feature extraction complete.
Shape of extracted features: (30, 1280)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import joblib

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded, num_classes=9)  # Adjust num_classes based on your dataset

print("Label encoding complete.")
print("Shape of one-hot encoded labels:", y_one_hot.shape)

# Save the label encoder to disk
joblib.dump(label_encoder, 'label_encoder.pkl')

Label encoding complete.
Shape of one-hot encoded labels: (30, 9)


['label_encoder.pkl']

In [ ]:
from sklearn.model_selection import train_test_split

# First split: train and temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y_one_hot, test_size=0.4, random_state=42)

# Second split: validation and test from temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Data splitting complete.")
print("Shape of training data:", X_train.shape, y_train.shape)
print("Shape of validation data:", X_val.shape, y_val.shape)
print("Shape of test data:", X_test.shape, y_test.shape)

Data splitting complete.
Shape of training data: (18, 1280) (18, 9)
Shape of validation data: (6, 1280) (6, 9)
Shape of test data: (6, 1280) (6, 9)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))  # Adjust num_classes based on your dataset

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 2.6458 - accuracy: 0.1667 - val_loss: 2.3504 - val_accuracy: 0.1667
Epoch 2/50
1/1 [==============================] - 0s 36ms/step - loss: 2.5793 - accuracy: 0.1667 - val_loss: 2.3313 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 36ms/step - loss: 2.4726 - accuracy: 0.3333 - val_loss: 2.0266 - val_accuracy: 0.1667
Epoch 4/50
1/1 [==============================] - 0s 52ms/step - loss: 2.2981 - accuracy: 0.3333 - val_loss: 1.7287 - val_accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 40ms/step - loss: 1.8895 - accuracy: 0.5000 - val_loss: 1.5309 - val_accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 36ms/step - loss: 1.6658 - accuracy: 0.3889 - val_loss: 1.3882 - val_accuracy: 0.3333
Epoch 7/50
1/1 [==============================] - 0s 38ms/step - loss: 1.7851 - accuracy: 0.2222 - val_loss: 1.3124 - val_accuracy: 0.3333
Epoch 8/50
1/1 [=========

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy}')


1/1 [==============================] - 0s 38ms/step - loss: 0.7936 - accuracy: 0.8333
Validation Accuracy: 0.8333333134651184


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


1/1 [==============================] - 0s 54ms/step - loss: 1.3544 - accuracy: 0.8333
Test Accuracy: 0.8333333134651184


In [ ]:
model.save('sign_language_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
